In [1]:
import os
# garbage_collection_threshold between 0 and 1 in % of mem
# backend native or cudaMallocAsync
# PYTORCH_CUDA_ALLOC_CONF=<option>:<value>,<option2>:<value2>...
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = f"garbage_collection_threshold:0.99"

In [3]:
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> {"type": "function", "function": {"name": "get_stock_fundamentals", "description": "get_stock_fundamentals(symbol: str) -> dict - Get fundamental data for a given stock symbol using yfinance API.\\n\\n    Args:\\n        symbol (str): The stock symbol.\\n\\n    Returns:\\n        dict: A dictionary containing fundamental data.\\n            Keys:\\n                - \'symbol\': The stock symbol.\\n                - \'company_name\': The long name of the company.\\n                - \'sector\': The sector to which the company belongs.\\n                - \'industry\': The industry to which the company belongs.\\n                - \'market_cap\': The market capitalization of the company.\\n                - \'pe_ratio\': The forward price-to-earnings ratio.\\n                - \'pb_ratio\': The price-to-book ratio.\\n                - \'dividend_yield\': The dividend yield.\\n                - \'eps\': The trailing earnings per share.\\n                - \'beta\': The beta value of the stock.\\n                - \'52_week_high\': The 52-week high price of the stock.\\n                - \'52_week_low\': The 52-week low price of the stock.", "parameters": {"type": "object", "properties": {"symbol": {"type": "string"}}, "required": ["symbol"]}}}  </tools> Use the following pydantic model json schema for each tool call you will make: {"properties": {"arguments": {"title": "Arguments", "type": "object"}, "name": {"title": "Name", "type": "string"}}, "required": ["arguments", "name"], "title": "FunctionCall", "type": "object"} For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{"arguments": <args-dict>, "name": <function-name>}
</tool_call>

SyntaxError: invalid syntax (1137638757.py, line 1)

In [2]:
from flask import Flask, render_template, request, jsonify
import requests
import torch

In [3]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from conversation import conv_templates, SeparatorStyle

from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

from llama_cpp import Llama

from PIL import Image
import requests
import torch
import copy
import json
import time

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
initial_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0])

In [5]:
print(initial_memory)

434307072


In [6]:
model = Llama(
    model_path="/home/tobi/ai/models/text_to_text/Meta-Llama-3-70B-Instruct-IQ2_S.gguf",
    n_gpu_layers=-1, # Uncomment to use GPU acceleration
    # seed=1337, # Uncomment to set a specific seed
    n_ctx=1024, # Uncomment to increase the context window
)

llama_model_loader: loaded meta data with 26 key-value pairs and 723 tensors from /home/tobi/ai/models/text_to_text/Meta-Llama-3-70B-Instruct-IQ2_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-70B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 80
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 8192
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 28672
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 64
llama_model_loader: - kv

In [7]:
model_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0]) - initial_memory
print(f"Model memory: {model_memory / 1024 / 1024:.2f} MB", model_memory)

Model memory: 22640.00 MB 23739760640


In [8]:
before_forward_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0])

In [13]:
output = model(
      "Q: Name the planets in the solar system? A: ", # Prompt
      max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=False # Echo the prompt back in the output
) # Generate a completion, can also call create_completion

Llama.generate: prefix-match hit

llama_print_timings:        load time =     614.86 ms
llama_print_timings:      sample time =       7.55 ms /    17 runs   (    0.44 ms per token,  2251.66 tokens per second)
llama_print_timings: prompt eval time =     351.08 ms /    13 tokens (   27.01 ms per token,    37.03 tokens per second)
llama_print_timings:        eval time =     458.97 ms /    16 runs   (   28.69 ms per token,    34.86 tokens per second)
llama_print_timings:       total time =     920.16 ms /    29 tokens


In [10]:
after_forward_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0])
token_memory = after_forward_memory - before_forward_memory
print(f"Memory per token: {token_memory / 1024 / 1024:.2f} MB", token_memory)

Memory per token: 498.00 MB 522190848


In [14]:
print(output)

{'id': 'cmpl-cd9b635a-65d9-4836-b070-0e9a417291e3', 'object': 'text_completion', 'created': 1716111721, 'model': '/home/tobi/ai/models/text_to_text/Meta-Llama-3-70B-Instruct-IQ2_S.gguf', 'choices': [{'text': ' Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus and Neptune', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 14, 'completion_tokens': 17, 'total_tokens': 31}}


In [23]:
print(json.dumps(torch.cuda.memory_stats_as_nested_dict(), indent=4))

{
    "num_alloc_retries": 0,
    "num_ooms": 0,
    "max_split_size": -1,
    "allocation": {
        "all": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "small_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "large_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        }
    },
    "segment": {
        "all": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "small_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "large_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        }
    },
    "active": {
        "all": {
            "current": 0,
            "peak": 0

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_init_text')
def get_init_text():
    returned_content = [
        {'role': 'System', 'content': 'Hello, I am the system.'},
    ]
    return jsonify({'status': 'success', 'returned_content': returned_content})

@app.route('/send_user_msg', methods=['POST'])
def send_user_msg():
    data = request.get_json()
    user = data['user']
    content = data['content']

    url = f"http://127.0.0.1:10000/infer"
    data_to_send = {}
    data_to_send['content'] = content
    data_to_send['use_image'] = False
    response = requests.post(url, json=data_to_send)    
    ai_reply = response.json()['returned_content']
    
    returned_content = [
        {'role': 'User', 'content': content},
        {'role': 'AI', 'content': "\n".join(ai_reply)}
    ]
    print(f"Received text from {user}: {content}")
    return jsonify({'status': 'success', 'returned_content': returned_content})

if __name__ == '__main__':
    app.run(port=14000)